In [1]:
import sys, os
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

sys.path.append('/content/drive/My Drive/inli_allennlp')
os.chdir('drive/My Drive/inli_allennlp')

Mounted at /content/drive


In [ ]:
! pip install allennlp

     |████████████████████████████████| 481kB 2.8MB/s 
     |████████████████████████████████| 266kB 12.7MB/s 
     |████████████████████████████████| 675kB 14.0MB/s 
     |████████████████████████████████| 317kB 13.2MB/s 
     |████████████████████████████████| 890kB 17.7MB/s 
     |████████████████████████████████| 1.1MB 23.8MB/s 
     |████████████████████████████████| 3.8MB 30.3MB/s 
  Created wheel for jsonnet: filename=jsonnet-0.16.0-cp36-cp36m-linux_x86_64.whl size=3321574 sha256=8a1157b1a5c32c7521f21e45500e65c88166189d762de29292b279a5af5f11cd
  Stored in directory: /root/.cache/pip/wheels/64/a9/43/bc5e0463deeec89dfca928a2a64595f1bdb520c891f6fbd09c
  Created wheel for overrides: filename=overrides-3.0.0-cp36-none-any.whl size=5669 sha256=bed59149eb6e8a5fbcbd779a5f0b5257589f2a068becbea4cf95554e819b980e
  Stored in directory: /root/.cache/pip/wheels/6f/1b/ec/6c71a1eb823df7f850d956b2d8c50a6d49c191e1063d73b9be
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.

In [3]:
import torch
from transformers import T5Tokenizer, T5Model, T5ForConditionalGeneration, Trainer, TrainingArguments
from models.model import IsnliModel
from allennlp.data import Vocabulary 
from allennlp.models.archival import load_archive

pretrained_name = 'base_5_epochs'
models_name = 't5-base'
archive = load_archive(f'results/snli/{pretrained_name}/model.tar.gz')
model = archive.model
tokenizer = T5Tokenizer.from_pretrained(models_name)


wandb: WARNING W&B installed but not logged in.  Run `wandb login` or set the WANDB_API_KEY env variable.
Some weights of T5ForConditionalGeneration were not initialized from the model checkpoint at t5-base and are newly initialized: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight', 'lm_head.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from collections import OrderedDict
import json
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
test_file_path = 'datasets/snli_1.0/snli_1.0_test.jsonl'
labelings = {"entailment":1, "neutral":2, "contradiction":0}
data = []
model = model.to(device)
with open(test_file_path) as jsonl_file:
  dataset_list = list(jsonl_file)
  for i, data_chunk in enumerate(dataset_list):
    data_chunk = json.loads(data_chunk)
    premise = data_chunk['sentence1']
    label_name = data_chunk['gold_label']
    if label_name == '-':
      continue
    label = labelings[label_name]
    input_sen = f"isnli {label_name}: {premise} {tokenizer.eos_token}"
    input_ids = tokenizer.encode(input_sen, return_tensors='pt').to(device)
    predicted_ids = model.generate(input_ids=input_ids)
    predicted_hypothesis = tokenizer.decode(predicted_ids[0])
    example = (OrderedDict([("premise", premise), ("hypothesis", predicted_hypothesis)]), label)
    data.append(example)
  print(data)

[(OrderedDict([('premise', 'This church choir sings to the masses as they sing joyous songs from the book at a church.'), ('hypothesis', 'The choir is singing for a large crowd.')]), 2), (OrderedDict([('premise', 'This church choir sings to the masses as they sing joyous songs from the book at a church.'), ('hypothesis', 'The choir is singing.')]), 1), (OrderedDict([('premise', 'This church choir sings to the masses as they sing joyous songs from the book at a church.'), ('hypothesis', 'The choir is singing in a bar.')]), 0), (OrderedDict([('premise', 'A woman with a green headscarf, blue shirt and a very big grin.'), ('hypothesis', 'A woman is grinning at her friend.')]), 2), (OrderedDict([('premise', 'A woman with a green headscarf, blue shirt and a very big grin.'), ('hypothesis', 'A woman is grinning.')]), 1), (OrderedDict([('premise', 'A woman with a green headscarf, blue shirt and a very big grin.'), ('hypothesis', 'A woman is frowning.')]), 0), (OrderedDict([('premise', 'An old 

In [8]:
import json
test_file_path = 'datasets/snli_1.0/snli_1.0_test.jsonl'

print("index ; premise ; label ; generated hypothesis ; ground truth")
with open(test_file_path) as jsonl_file:
  dataset_list = list(jsonl_file)
  for i, data_chunk in enumerate(dataset_list):
    data_chunk = json.loads(data_chunk)
    premise = data_chunk['sentence1']
    hypothesis = data_chunk['sentence2']
    label = data_chunk['gold_label']
    if label == '-':
      i -= 1
      continue
    input_sen = f"isnli {label}: {premise} {tokenizer.eos_token}"
    input_ids = tokenizer.encode(input_sen, return_tensors='pt')
    predicted_ids = model.generate(input_ids=input_ids)
    print(f'{i} ; {premise} ; {label} ; {tokenizer.decode(predicted_ids[0])} ; {hypothesis}')
    if i == 200:
      break

index ; premise ; label ; generated hypothesis ; ground truth
0 ; This church choir sings to the masses as they sing joyous songs from the book at a church. ; neutral ; The choir is singing for a large crowd. ; The church has cracks in the ceiling.
1 ; This church choir sings to the masses as they sing joyous songs from the book at a church. ; entailment ; The choir is singing. ; The church is filled with song.
2 ; This church choir sings to the masses as they sing joyous songs from the book at a church. ; contradiction ; The choir is singing in a bar. ; A choir singing at a baseball game.
3 ; A woman with a green headscarf, blue shirt and a very big grin. ; neutral ; A woman is grinning at her friend. ; The woman is young.
4 ; A woman with a green headscarf, blue shirt and a very big grin. ; entailment ; A woman is grinning. ; The woman is very happy.
5 ; A woman with a green headscarf, blue shirt and a very big grin. ; contradiction ; A woman is frowning. ; The woman has been shot.
6

In [9]:
!pip install textattack

In [10]:
!pip uninstall pyarrow
!pip install pyarrow===0.16.0

Uninstalling pyarrow-0.16.0:
  Would remove:
    /usr/local/bin/plasma_store
    /usr/local/lib/python3.6/dist-packages/pyarrow-0.16.0.dist-info/*
    /usr/local/lib/python3.6/dist-packages/pyarrow/*
  Would not remove (might be manually added):
    /usr/local/lib/python3.6/dist-packages/pyarrow/_orc.cpp
    /usr/local/lib/python3.6/dist-packages/pyarrow/_orc.cpython-36m-x86_64-linux-gnu.so
    /usr/local/lib/python3.6/dist-packages/pyarrow/include/arrow/adapters/orc/adapter.h
    /usr/local/lib/python3.6/dist-packages/pyarrow/include/arrow/allocator.h
    /usr/local/lib/python3.6/dist-packages/pyarrow/include/arrow/buffer-builder.h
    /usr/local/lib/python3.6/dist-packages/pyarrow/include/arrow/compute/benchmark-util.h
    /usr/local/lib/python3.6/dist-packages/pyarrow/include/arrow/compute/test-util.h
    /usr/local/lib/python3.6/dist-packages/pyarrow/include/arrow/csv/column-builder.h
    /usr/local/lib/python3.6/dist-packages/pyarrow/include/arrow/csv/test-common.h
    /usr/local/

In [11]:
!textattack eval --model bert-base-uncased-snli --dataset-from-file datasets/snli_dataset_list2.py --num-examples 9824 --batch-size 1

2020-07-28 16:07:48.347870: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
wandb: WARNING W&B installed but not logged in.  Run `wandb login` or set the WANDB_API_KEY env variable.
textattack: Loading pre-trained model from HuggingFace model repository: textattack/bert-base-uncased-snli
textattack: Loading model and tokenizer from file: None
textattack: Loading module from `datasets/snli_dataset_list2.py`.
textattack: Got 9824 predictions.
textattack: Successes 7965/9824 (81.08%)
